# Installing Security Telephones

This is Example 9.1-2 from [Operations Research, 8th Edition by Taha](https://www.amazon.com/Operations-Research-Introduction-Hamdy-Taha/dp/0131889230/ref=sr_1_1?ie=UTF8&qid=1537392218&sr=8-1&keywords=operations+research+taha+8).

To promote on-campus safety, the U of A Security Department is in the process of installing emergency telephones at selected locations. The department wants to install the minimum number of telephones, provided that each of the campus main streets is served by at least one telephone. The following pictures maps the principal streets (A to K) on campus.

<img src="./installing_security_telephones.PNG" style="width: 600px"><br/>

This problem is a set-covering problem because we want to determine the minimum number of installations that need to be covered for each facility.

It is logical to place the telephones at street intersections so that each telephone will serve at least two streets. Figure 9.1 shows that the layout of the streets requires a maximum of eight telephone locations.

The optimum solution of the problem requires installing four telephones at intersections 1,2,5, and 7.

## Formulation

$$ min \enspace z = x_1 + x_2 + x_3 + x_4 + x_5 + x_6 + x_7 + x_8 $$ 
$$ \\ x_1 + x_2 \geq 1 $$ 
$$ \\ x_2 + x_3 \geq 1 $$
$$ \\ x_4 + x_5 \geq 1 $$ 
$$ \\ x_7 + x_8 \geq 1 $$ 
$$ \\ x_6 + x_7 \geq 1 $$ 
$$ \\ x_2 + x_6 \geq 1 $$ 
$$ \\ x_1 + x_6 \geq 1 $$ 
$$ \\ x_4 + x_7 \geq 1 $$ 
$$ \\ x_2 + x_4 \geq 1 $$ 
$$ \\ x_5 + x_8 \geq 1 $$ 
$$ \\ x_3 + x_5 \geq 1 $$ 
$$ \\ x_j = (0, 1), j = 1, 2, \dots, 8 $$ <br/> 

## 1. Import Packages

In [1]:
from __future__ import print_function
import sys
import cplex
from cplex.exceptions import CplexError

## 2. Data

In [2]:
import numpy as np

A = [[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
     [0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0],
     [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0],
     [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0],
     [0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
     [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
     [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0],
     [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
     [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0],
     [0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]]
b = [1.0] * len(A)
c = [1.0] * len(A[0])
u = [cplex.infinity] * len(A[0])
l = [0.0] * len(A[0])
# Optimization objective sense can be 'min' or 'max'
optSense = 'min'  
varType = "I" * len(A[0])
saveOpt = False
quietOpt = True

## 3. Code

In [3]:
def mip(A, b, c, u, l, varType, optSense, saveOpt, quietOpt):

    noRow, noCol = len(A), len(A[0])
    
    my_prob = cplex.Cplex()
    if optSense == 'min':
        my_prob.objective.set_sense(my_prob.objective.sense.minimize)
    elif optSense == 'max':
        my_prob.objective.set_sense(my_prob.objective.sense.maximize)

    my_colnames = ["x"+str(i) for i in range(noCol)]
    # for senses, G, L, E, R means greater-than, less-than, equality, ranged constraints
    my_prob.variables.add(obj=c, ub=u, lb=l, types=varType, names=my_colnames)
    
    A = [[[i for i in range(noCol)], A[j]] for j in range(noRow)]
    my_prob.linear_constraints.add(lin_expr=A, senses=["G" * noRow], rhs=b)
    
    if quietOpt:
        my_prob.set_log_stream(None)
        my_prob.set_error_stream(None)
        my_prob.set_warning_stream(None)
        my_prob.set_results_stream(None)
    
    try:
        my_prob.solve()
        if saveOpt:
            my_prob.write("Installing Security Telephones.lp")
            print('LP saved as Installing Security Telephones.lp')

        x = my_prob.solution.get_values()
        obj = my_prob.solution.get_objective_value()
        return x, obj
    except CplexError as exc:
        print(exc)
        return [], -1

In [4]:
%%time

x, obj = mip(A, b, c, u, l, varType, optSense, saveOpt, quietOpt)
print(x)
print(obj)

[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0]
4.0
CPU times: user 5.83 ms, sys: 5.47 ms, total: 11.3 ms
Wall time: 9.15 ms


What if we change the variable type to continuous type 'C'?

In [5]:
%%time 

varType = 'C' * len(A[0])
x, obj = mip(A, b, c, u, l, varType, optSense, saveOpt, quietOpt)
print(x)
print(obj)

[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0]
4.0
CPU times: user 9.32 ms, sys: 1.84 ms, total: 11.2 ms
Wall time: 10.2 ms


Since all the input this time is integer, therefore we get integer output as well. If we change just one number for the input to a decimal number, we can't get our desired result.

In [6]:
%%time 

c[1] = 1.1
x, obj = mip(A, b, c, u, l, varType, optSense, saveOpt, quietOpt)
print(x)
print(obj)

[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
4.05
CPU times: user 9.73 ms, sys: 2.2 ms, total: 11.9 ms
Wall time: 10.5 ms
